In [2]:
from datetime import datetime
import pandas as pd
import os
from os.path import join

In [45]:
def 윤년판단(year) : 
    return (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0)

def 일_연체이자(시기, 종기, 연체이자율, 미상환원금) : 
    당해총일수 = 366 if 윤년판단(시기.year) else 365
    이자발생일수 = (종기 - 시기).days + 1  # 종기도 이자발생일이므로
    return 미상환원금 * (연체이자율 / 당해총일수) * 이자발생일수

def 연_연체이자(연도수, 연체이자율, 미상환원금) :
    return 미상환원금 * 연체이자율 * 연도수

def 연체이자계산(시기, 종기, 연체이자율, 미상환원금) :
    """ 
    시기 : 연체이자 발생 첫날, yyyy-mm-dd형식의 문자열이거나 날짜
    종기 : 연체이자 발생 마지막날, yyyy-mm-dd형식의 문자열이거나 날짜
    계산방식 : 연이자 적용, 연이자 불가할 경우 일이자(윤년의 경우 366으로 나눔), 최종날짜의 소수점 이하는 버림
    (나우리는 일이자를 원단위 이하 버리고 어제의 연체이자에 합산하는 식으로 하는듯. 그래서 연이자가 적음)
    """
    date_format = "%Y-%m-%d"
    if not isinstance(시기, datetime) : 시기 = datetime.strptime(시기, date_format)
    if not isinstance(종기, datetime) : 종기 = datetime.strptime(종기, date_format)
    
    연체이자율 = float(연체이자율) / 100 # 문자열이거나 정수로 input 될 수 있으니
    미상환원금 = 미상환원금
    
    # 시기와 종기가 같은 해인경우
    if 시기.year == 종기.year : 
        return int(일_연체이자(시기, 종기, 연체이자율, 미상환원금))
    else : 
        첫해연체이자 = 일_연체이자(시기, datetime(시기.year, 12, 31), 연체이자율, 미상환원금)
        중간연체이자 = 연_연체이자(종기.year - 시기.year -1, 연체이자율, 미상환원금)
        마지막해연체이자 = 일_연체이자(datetime(종기.year, 1, 1), 종기, 연체이자율, 미상환원금)
        return int(첫해연체이자 + 중간연체이자 + 마지막해연체이자)

In [42]:
dir = r"D:\3.자산\프로젝트\2023 전산데이터 클렌징 및 업무개편 프로젝트\대성\대성 DC(대정,솔림01)\전산 수정 완료(LOG)"
fn = "240426_대성_계좌_최초이자 재계산.xlsx"

df = pd.read_excel(join(dir, fn), dtype={"계좌키":str, "타계좌키":str})

In [46]:
df["수정최초미수이자"] = df.apply(lambda row: 연체이자계산(row['솔림최초이자기준일'], row['컷오프일'], row['연체이자율'], row['최초원금']), axis=1)

In [48]:
df.to_excel(join(dir, "추가미수이자.xlsx"), index = False)

In [50]:
df.컷오프일.values[0]

'2021-11-23'